In [45]:
from bokeh.layouts import column,row
from bokeh.models import ColumnDataSource, CustomJS, Slider, Select
from bokeh.plotting import Figure, output_file, save
from bokeh.models.widgets import Select,Button
from numpy.random import randint
from bokeh import events

def index_containing_substring(the_list, substring):
    for i, s in enumerate(the_list):
        if substring in s:
              return i
    return -1


x = [x for x in range(-10, 10)]
y = x
label = list(randint(0,2,len(x)))
print(label)

cluster = 1

select_dataset = Select(title="Select a dataset:", value="a", options=['a','b','c'],width=100)
select_cluster = Select(title="Selected cluster:", value="1", options=['%s'%(i)for i in range(1,10)])
source = ColumnDataSource(data=dict(x=x, y=y,label=label, color =['red']*20))
button = Button(label="Expain it to me!")

TOOLS = "box_select,lasso_select"
plot = Figure(tools = TOOLS,title='Selected dataset')
plot.scatter('x', 'y', source=source,color='color')

select_dataset.js_on_change("value", CustomJS(args=dict(title= plot.title,source=source),code="""
    title.text = this.value;
    d3.json("/get_dataset_"+this.value,function(data){
        source.data['x'] = Object.values(data['x']);
        source.data['y'] = Object.values(data['y']);
        source.data['label'] = Object.values(data['label']);
        source.data['color'] = Object.values(data['color']);
        source.change.emit();
    });    
"""))
select_cluster.js_on_change("value", CustomJS(args=dict(source=source),code="""
    cl = +this.value;
"""))
source.selected.js_on_change('indices',CustomJS(args=dict(source=source),code="""
    console.log("source changed");
    var data = source.data;
    cb_obj.indices.forEach(element =>(data.color[element]=col(cl),data.label[element]=cl))
    source.change.emit();
"""))
button.js_on_event(events.ButtonClick,CustomJS(args=dict(source=source),code="""
    console.log("Button pressed");
    d3.json("/elaborate_dataset?dataset="+JSON.stringify(source.data['label']).replaceAll('#','%23'),function(data){
        visualize_explanation(data.src);
    });
"""))


layout = row(select_dataset,column(select_cluster,plot,button))
layout.sizing_mode = 'scale_width'
output_file("templates/scatter_plot.html", title='Bokeh Plot')
save(layout)

f = open("templates/scatter_plot.html", "r")
contents = f.readlines()
f.close()

index = index_containing_substring(contents, "<head>")
f = open("templates/d3_code_head.txt", "r")
hard_code = f.readlines()
f.close()
contents.insert(index+1,''.join(hard_code));


index = index_containing_substring(contents, "<body>")
f = open("templates/d3_code_body.txt", "r")
hard_code = f.readlines()
f.close()
contents.insert(index+1,''.join(hard_code));



#contents.insert(index+1,"col = d3.scaleOrdinal().domain([1,10]).range(d3.schemeSet1)</script>");

f = open("templates/scatter_plot.html", "w")
contents = "".join(contents)
f.write(contents)
f.close()

[1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1]
